<a href="https://colab.research.google.com/github/sanjeetrk/yt-video-downloader-tg-bot/blob/main/both_search_audio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import requests
import re
from google.colab import userdata
try:
    import telebot
    from pytube import YouTube
except ImportError:
    !pip install pyTelegramBotAPI pytube
    import telebot
    from pytube import YouTube

# Initialize your bot with the Telegram token
try:
  TOKEN = userdata.get('ytdownkey')
except userdata.SecretNotFoundError:
  TOKEN=input("Enter your Bot token\n")
bot = telebot.TeleBot(TOKEN)

def download_audio(url, output_dir):
    yt = YouTube(url)
    audio_stream = yt.streams.filter(only_audio=True).order_by('abr').desc().first()
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    audio_file_path = audio_stream.download(output_dir)
    new_file_path = audio_file_path[:-4] + ".mp3"
    os.rename(audio_file_path, new_file_path)
    return new_file_path

def search_youtube_videos(query):
    encoded_query = requests.utils.quote(query)
    search_url = f"https://www.youtube.com/results?search_query={encoded_query}"
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(search_url, headers=headers)
    # Matching video IDs and Titles from the page
    # video_ids = re.findall(r'watch\?v=(\S{11})', response.text)
    video_ids = re.findall(r'watch\?v=([a-zA-Z0-9_-]{11})', response.text)

    video_titles = re.findall(r'"title":{"runs":\[{"text":"([^"]+)"\}\]', response.text)
    links = []
    for i in range(min(len(video_ids), 5)):  # Collect up to 5 video links and titles
        video_url = f"https://www.youtube.com/watch?v={video_ids[i]}"
        video_title = video_titles[i] if i < len(video_titles) else "No title available"
        links.append(f"{video_title} - {video_url}\n")
    return links

@bot.message_handler(commands=['start'])
def send_welcome(message):
    bot.reply_to(message, "Send me a YouTube link to download audio, or any text to search on YouTube.")

@bot.message_handler(func=lambda message: True)
def handle_message(message):
    if 'http' in message.text or 'https' in message.text:
        try:
            video_url = message.text
            output_directory = "output/"
            audio_file_path = download_audio(video_url, output_directory)
            with open(audio_file_path, 'rb') as audio_file:
                bot.send_audio(message.chat.id, audio_file)
            os.remove(audio_file_path)
        except Exception as e:
            bot.reply_to(message, f"Error downloading audio: {str(e)}")
    else:
        try:
            search_results = search_youtube_videos(message.text)
            response_message = "Top search results:\n" + "\n".join(search_results)
            bot.send_message(message.chat.id, response_message)
        except Exception as e:
            bot.reply_to(message, f"Error searching YouTube: {str(e)}")

bot.polling()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.9/242.9 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 7.4 MB/s eta 0:00:00
